In [6]:
import os
import nltk

# Définir le chemin local pour nltk_data (dans ton dossier de projet)
nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')

# Créer le dossier s'il n'existe pas
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)

# Forcer NLTK à utiliser ce dossier en le plaçant en tête de la liste
os.environ['NLTK_DATA'] = nltk_data_path
if nltk_data_path not in nltk.data.path:
    nltk.data.path.insert(0, nltk_data_path)

# Télécharger les ressources nécessaires, y compris 'punkt_tab'
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('punkt_tab', download_dir=nltk_data_path)  # <-- Ajout pour résoudre le LookupError
nltk.download('gutenberg', download_dir=nltk_data_path)

print("Chemin nltk_data utilisé :", nltk_data_path)
print("Liste des chemins recherchés par NLTK :", nltk.data.path)


Chemin nltk_data utilisé : c:\Users\ghali\Documents\projet_autocorrect\nltk_data
Liste des chemins recherchés par NLTK : ['c:\\Users\\ghali\\Documents\\projet_autocorrect\\nltk_data', 'C:\\Users\\ghali/nltk_data', 'c:\\Users\\ghali\\anaconda3\\nltk_data', 'c:\\Users\\ghali\\anaconda3\\share\\nltk_data', 'c:\\Users\\ghali\\anaconda3\\lib\\nltk_data', 'C:\\Users\\ghali\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\ghali\Documents\projet_autocorrect\nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\ghali\Documents\projet_autocorrect\nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     c:\Users\ghali\Documents\projet_autocorrect\nltk_data.
[nltk_data]     ..
[nltk_data]   Package gutenberg is already up-to-date!


In [7]:
try:
    nltk.data.find('tokenizers/punkt/english.pickle')
    print("Le fichier english.pickle est trouvé !")
except LookupError:
    print("Le fichier english.pickle est introuvable. Réessaie le téléchargement.")


Le fichier english.pickle est trouvé !


In [8]:
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize

# Charger le texte du livre "Emma" de Jane Austen
texte = gutenberg.raw('austen-persuasion.txt')

# Tokeniser le texte en le mettant en minuscule
tokens = word_tokenize(texte.lower())

# Garder uniquement les mots alphabétiques
mots = [mot for mot in tokens if mot.isalpha()]

# Créer un vocabulaire unique (ensemble des mots)
vocabulaire = set(mots)

print("Nombre total de mots :", len(mots))
print("Nombre de mots uniques :", len(vocabulaire))


Nombre total de mots : 83034
Nombre de mots uniques : 5653


In [9]:
import difflib

def generer_candidats(mot, vocabulaire, n=3):
    """
    Retourne jusqu'à n candidats proches pour le mot donné.
    """
    return difflib.get_close_matches(mot, vocabulaire, n=n)


In [10]:
from collections import Counter

# Calculer la fréquence de chaque mot dans le corpus
frequences = Counter(mots)

def meilleure_correction(mot, vocabulaire, frequences):
    """
    Retourne la correction la plus probable pour un mot mal orthographié,
    en choisissant parmi les candidats celui qui a la fréquence la plus élevée.
    """
    candidats = generer_candidats(mot, vocabulaire)
    if not candidats:
        return mot  # Aucun candidat trouvé, on retourne le mot original
    return max(candidats, key=lambda c: frequences[c])


In [11]:
mot_test = "definately"
correction = meilleure_correction(mot_test, vocabulaire, frequences)
print(f"Correction de '{mot_test}' => '{correction}'")


Correction de 'definately' => 'infinitely'


In [13]:
def corriger_phrase(phrase, vocabulaire, frequences):
    tokens = word_tokenize(phrase.lower())
    # Pour chaque mot, si c'est alphabétique, on applique la correction
    mots_corriges = [
        meilleure_correction(mot, vocabulaire, frequences) if mot.isalpha() else mot
        for mot in tokens
    ]
    return ' '.join(mots_corriges)

# Exemple de test sur une phrase
exemple = "I definately recieve teh best gifts"
print("Avant :", exemple)
print("Après :", corriger_phrase(exemple, vocabulaire, frequences))


Avant : I definately recieve teh best gifts
Après : i infinitely receive the best its


In [14]:
from nltk.util import ngrams

# Construire les bigrammes à partir de la liste "mots"
bigram_counts = Counter(ngrams(mots, 2))
unigram_counts = Counter(mots)

def bigram_probability(prev, word):
    """
    Calcule la probabilité P(word|prev) = count(prev, word) / count(prev)
    """
    if unigram_counts[prev] == 0:
        return 0
    return bigram_counts[(prev, word)] / unigram_counts[prev]


In [15]:
def meilleure_correction_context(mot, vocabulaire, frequences, prev_word=None):
    """
    Correction d'un mot en tenant compte du contexte (mot précédent).
    Si prev_word est fourni, on évalue chaque candidat avec :
        score = bigram_probability(prev_word, candidat) * frequences[candidat]
    Sinon, on utilise la fonction de correction de base.
    """
    candidats = generer_candidats(mot, vocabulaire)
    if not candidats:
        return mot
    if prev_word is None:
        return max(candidats, key=lambda c: frequences[c])
    
    # Calculer le score pour chaque candidat
    scores = {}
    for candidat in candidats:
        # On combine la probabilité bigramme avec la fréquence (pour différencier les candidats)
        prob = bigram_probability(prev_word, candidat)
        scores[candidat] = prob * frequences[candidat]
    
    # Si tous les scores sont nuls, on retombe sur la fréquence seule
    if all(score == 0 for score in scores.values()):
        return max(candidats, key=lambda c: frequences[c])
    
    return max(scores, key=scores.get)


In [17]:
def corriger_phrase_context(phrase, vocabulaire, frequences):
    tokens = word_tokenize(phrase.lower())
    mots_corriges = []
    prev_word = None
    for mot in tokens:
        if mot.isalpha():
            # Utilise la correction contextuelle
            correction = meilleure_correction_context(mot, vocabulaire, frequences, prev_word)
            mots_corriges.append(correction)
            prev_word = correction  # On utilise le mot corrigé pour le contexte suivant
        else:
            mots_corriges.append(mot)
            prev_word = None  # Reset si on a de la ponctuation
    return ' '.join(mots_corriges)

# Test de la fonction avec contexte
exemple_context = "I definately recieve teh best gifts"
print("Avant :", exemple_context)
print("Après :", corriger_phrase_context(exemple_context, vocabulaire, frequences))


Avant : I definately recieve teh best gifts
Après : i infinitely receive the best its


In [19]:
def load_corpus(lang="en"):
    """
    Charge un corpus en fonction de la langue.
    Pour "en", on utilise le corpus Gutenberg (austen-emma.txt).
    Pour "fr", on charge le fichier "corpus_fr.txt" (à créer par l'utilisateur).
    """
    if lang == "en":
        from nltk.corpus import gutenberg
        texte = gutenberg.raw('austen-emma.txt')
    elif lang == "fr":
        try:
            with open("corpus_fr.txt", "r", encoding="utf-8") as f:
                texte = f.read()
        except FileNotFoundError:
            raise FileNotFoundError("Le fichier corpus_fr.txt n'a pas été trouvé dans le dossier du projet.")
    else:
        raise ValueError("Langue non supportée. Choisir 'en' ou 'fr'.")
    return texte


In [20]:
from nltk.tokenize import word_tokenize
from collections import Counter

def build_vocabulary(texte, language="en"):
    """
    Tokenise le texte en utilisant le paramètre de langue, filtre les tokens et crée le vocabulaire.
    Retourne un ensemble de mots et un Counter des fréquences.
    """
    # Utilise "english" si lang == "en", sinon "french"
    lang_param = "english" if language=="en" else "french"
    tokens = word_tokenize(texte.lower(), language=lang_param)
    mots = [mot for mot in tokens if mot.isalpha()]
    vocabulaire = set(mots)
    frequences = Counter(mots)
    print(f"Corpus chargé en {language}: {len(mots)} mots, {len(vocabulaire)} uniques.")
    return vocabulaire, frequences


In [21]:
import difflib

def generer_candidats(mot, vocabulaire, n=3):
    return difflib.get_close_matches(mot, vocabulaire, n=n)

def meilleure_correction(mot, vocabulaire, frequences):
    candidats = generer_candidats(mot, vocabulaire)
    if not candidats:
        return mot
    return max(candidats, key=lambda c: frequences[c])

def corriger_phrase(phrase, vocabulaire, frequences, lang="en"):
    tokens = word_tokenize(phrase.lower(), language="english" if lang=="en" else "french")
    mots_corriges = [
        meilleure_correction(mot, vocabulaire, frequences) if mot.isalpha() else mot
        for mot in tokens
    ]
    return ' '.join(mots_corriges)


In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import difflib
from collections import Counter
from nltk.tokenize import word_tokenize
import threading
import nltk

# Auto download if not found
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/gutenberg')
except LookupError:
    nltk.download('gutenberg')

# Chargement du corpus
def load_corpus(lang="en"):
    if lang == "en":
        from nltk.corpus import gutenberg
        texte = gutenberg.raw('austen-emma.txt')
    elif lang == "fr":
        with open("corpus_fr.txt", "r", encoding="utf-8") as f:
            texte = f.read()
    else:
        raise ValueError("Langue non supportée.")
    return texte

def build_vocabulary(texte, language="en"):
    lang = "english" if language == "en" else "french"
    tokens = word_tokenize(texte.lower(), language=lang)
    mots = [mot for mot in tokens if mot.isalpha()]
    return set(mots), Counter(mots)

def generer_candidats(mot, vocabulaire, n=3):
    return difflib.get_close_matches(mot, vocabulaire, n=n)

class AutoCorrectApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🔤 Correcteur Interactif NLP")
        self.root.geometry("900x600")

        self.vocabulaire = {}
        self.frequences = {}
        self.suggestion_vars = []

        self.build_gui()
        self.auto_load_corpus("en")

    def build_gui(self):
        top_frame = ttk.Frame(self.root)
        top_frame.pack(pady=10)

        ttk.Label(top_frame, text="Langue :").pack(side="left", padx=5)
        self.language = tk.StringVar(value="en")
        self.language_menu = ttk.OptionMenu(top_frame, self.language, "en", "en", "fr", command=self.auto_load_corpus)
        self.language_menu.pack(side="left", padx=5)

        self.status_label = ttk.Label(self.root, text="Chargement du corpus...", foreground="blue")
        self.status_label.pack(pady=5)

        self.text_input = tk.Text(self.root, width=90, height=7, font=("Arial", 13))
        self.text_input.pack(pady=10)

        self.correct_button = ttk.Button(self.root, text="🔧 Proposer des corrections", command=self.corriger_interactivement)
        self.correct_button.pack(pady=5)

        self.scroll_canvas = tk.Canvas(self.root)
        self.scroll_frame = ttk.Frame(self.scroll_canvas)
        self.scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=self.scroll_canvas.yview)
        self.scroll_canvas.configure(yscrollcommand=self.scrollbar.set)

        self.scroll_canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")
        self.canvas_frame = self.scroll_canvas.create_window((0, 0), window=self.scroll_frame, anchor='nw')
        self.scroll_frame.bind("<Configure>", lambda e: self.scroll_canvas.configure(scrollregion=self.scroll_canvas.bbox("all")))

        self.confirm_button = ttk.Button(self.root, text="✅ Confirmer les choix", command=self.confirmer_correction)
        self.confirm_button.pack(pady=10)

        self.result_label = ttk.Label(self.root, text="", wraplength=800, font=("Arial", 12), foreground="green")
        self.result_label.pack(pady=5)

    def auto_load_corpus(self, lang):
        def run():
            self.status_label.config(text="Chargement du corpus...", foreground="orange")
            texte = load_corpus(lang)
            self.vocabulaire[lang], self.frequences[lang] = build_vocabulary(texte, lang)
            self.status_label.config(text=f"✔ Corpus {lang.upper()} chargé ({len(self.vocabulaire[lang])} mots)", foreground="green")
        threading.Thread(target=run).start()

    def corriger_interactivement(self):
        for widget in self.scroll_frame.winfo_children():
            widget.destroy()
        self.suggestion_vars.clear()

        phrase = self.text_input.get("1.0", "end").strip()
        lang = self.language.get()
        tokens = word_tokenize(phrase.lower(), language="english" if lang == "en" else "french")

        for mot in tokens:
            frame = ttk.Frame(self.scroll_frame, padding=5)
            frame.pack(fill="x", pady=3)

            ttk.Label(frame, text=f"Mot : '{mot}'").pack(side="left", padx=5)
            var = tk.StringVar(frame)

            if mot.isalpha() and mot not in self.vocabulaire[lang]:
                candidats = generer_candidats(mot, self.vocabulaire[lang], n=3)
                if not candidats:
                    candidats = [mot]
                var.set(candidats[0])
                option_menu = ttk.OptionMenu(frame, var, candidats[0], *candidats)
                option_menu.pack(side="left", padx=10)
            else:
                var.set(mot)
                ttk.Label(frame, text="(Mot valide)").pack(side="left", padx=10)

            self.suggestion_vars.append(var)

    def confirmer_correction(self):
        corrected = [var.get() for var in self.suggestion_vars]
        phrase_corrigée = " ".join(corrected)
        self.result_label.config(text=f"🟢 Texte corrigé :\n{phrase_corrigée}")
        self.root.clipboard_clear()
        self.root.clipboard_append(phrase_corrigée)
        self.root.update()

# Lancement
if __name__ == "__main__":
    root = tk.Tk()
    app = AutoCorrectApp(root)
    root.mainloop()


Exception in thread Thread-5 (run):
Traceback (most recent call last):
  File "c:\Users\ghali\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\ghali\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ghali\AppData\Local\Temp\ipykernel_10836\2050975476.py", line 92, in run
  File "C:\Users\ghali\AppData\Local\Temp\ipykernel_10836\2050975476.py", line 34, in build_vocabulary
  File "c:\Users\ghali\anaconda3\Lib\site-packages\nltk\tokenize\__init__.py", line 142, in word_tokenize
    sentences = [text] if preserve_line else sent_tokenize(text, language)
                                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ghali\anaconda3\Lib\site-packages\nltk\tokenize\__init__.py", line 119, in sent_tokenize
    tokenizer = _get_punkt_tokenizer(language)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ghali\anaconda3\Lib\site-packages\nltk\tokenize\_